In [ ]:
# 초단기 실황 조회

In [ ]:
import requests
from datetime import datetime, timedelta
from pytz import timezone
import pandas as pd

def get_weather_data():
    while True:
        try:
            # 한국
            KST = timezone('Asia/Seoul')

            url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getUltraSrtNcst"
            service_key = "wO9XsANP1Nt9zBBV1WlwVtpfusq7uJZzkoRJjkfjIQxkgaf9X%2FbBCYOCgaS60vye6TNtlhmP1yhL%2Fm9NojPLFw%3D%3D"

            # 웹 요청할 base_date, base_time 계산
            now = datetime.now() # 현재 시각 데이터 생성
            now = now.astimezone(KST)

            # 40분 이전이면 현재 시보다 1시간 전 `base_time`을 요청
            if now.minute <= 40:
                # 단. 00:40분 이전이라면 `base_date`는 전날이고 `base_time`은 2300
                if now.hour == 0 :
                    base_date = (now-timedelta(days=1)).strftime('%Y%m%d')
                    base_time = '2300'
                else:
                    base_date = now.strftime('%Y%m%d')
                    base_time = (now-timedelta(hours=1)).strftime('%H00')
            # 40분 이후면 현재 시와 같은 `base_time`을 요청
            else:
                base_date = base_date = now.strftime('%Y%m%d')
                base_time = now.strftime('%H00')

            # 웹 요청시 같이 전달될 데이터 = 요청 메시지
            params = {
                'serviceKey' : service_key,
                'numOfRows' : 30,
                'pageNo' : 1,
                'dataType' : 'JSON',
                'base_date' : base_date,
                'base_time' : base_time,
                'nx' : 60, # 서울 기상관측소(종로구)
                'ny' : 127
            }

            res = requests.get(url=url , params=params)
            # res.raise_for_status()

            # response 데이터 정리
            data = res.json()['response']['body']['items']['item']


            categorys = {
                'T1H':'기온',
                'RN1':'1시간 강수량',
                'UUU':'동서바람성분',
                'VVV':'남북바람성분',
                'REH':'습도',
                'PTY':'강수형태',
                'VEC':'풍향',
                'WSD':'풍속',
            }

            # 최종 데이터 생성(dict)
            results = {}
            results['date'] = datetime.strptime(base_date+base_time, "%Y%m%d%H%M").strftime("%Y-%m-%d %H:%M:%S")
            for d in data:
                results[categorys[d['category']]] = d['obsrValue']

            # 필요한 데이터 추출 및 DF 변환
            filtered_data = {
                'date': results['date'],
                'temp': results['기온'],
                'mm': results['1시간 강수량'],
                'ws': results['풍속'],
                'humidity': results['습도']
            }
            convert_data = pd.DataFrame(filtered_data, index=[0])

            if not convert_data.empty:
                return convert_data

        except Exception as e:  # 통신 등의 이유로 에러 발생 시, 다시 시도
            print('Re-Try')

In [ ]:
# 실행
weather_data = get_weather_data()
print(weather_data)

                  date  temp mm   ws humidity
0  2024-05-11 23:00:00  14.1  0  0.7       93


In [ ]:
# pytorch(baseline)

In [ ]:
X.shape

torch.Size([17497, 24, 4])

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 임의의 데이터 생성
np.random.seed(0)
date_range = pd.date_range(start='2022-01-01', end='2024-01-01', freq='H')
data = pd.DataFrame({
    'date': date_range,
    'temp': np.random.uniform(0, 40, len(date_range)),  # 온도
    'mm': np.random.uniform(0, 20, len(date_range)),    # 강수량
    'ws': np.random.uniform(0, 10, len(date_range)),    # 풍속
    'humidity': np.random.uniform(0, 100, len(date_range))  # 습도
})

# 데이터를 정규화
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['temp', 'mm', 'ws', 'humidity']])

# 시계열 데이터를 LSTM 입력 형식으로 변환
def create_sequences(data, time_steps):
    X, Y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps, :])
        Y.append(data[i+time_steps, 0])  # 온도는 1번째 열
    return torch.tensor(X).float(), torch.tensor(Y).float()

time_steps = 24  # 24시간을 한 묶음으로 입력
X, Y = create_sequences(scaled_data, time_steps)

# 데이터셋 분할 (Train/Test)
split = int(0.8 * len(data))
X_train, X_test = X[:split], X[split:]
Y_train, Y_test = Y[:split], Y[split:]

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# 모델 초기화
input_size = X.shape[2]
hidden_size = 50
num_layers = 1
model = LSTMModel(input_size, hidden_size, num_layers)

# 손실 함수 및 최적화 함수
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Early stopping 관련 변수 설정
best_loss = float('inf')
patience = 5  # for early stopping
counter = 0

# 모델 훈련
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)
    loss.backward()
    optimizer.step()

    # 검증 데이터에 대한 손실 계산
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test)
        val_loss = criterion(val_outputs, Y_test)

    # 손실이 감소하면 모델 저장
    if val_loss < best_loss:
        best_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), 'best_model.pt')
    else:
        counter += 1

    # 조기 종료
    if counter >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# 모델 로드
model.load_state_dict(torch.load('best_model.pt'))

# 모델 평가
model.eval()
with torch.no_grad():
    predicted_temp = model(X_test).numpy()

# 예측값을 실제 값의 스케일로 변환
predicted_temp = scaler.inverse_transform(np.hstack((np.zeros((len(predicted_temp), 3)), np.array(predicted_temp).reshape(-1, 1)))).T[0]

# 예측값 출력
print(predicted_temp)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([14016])) that is different to the input size (torch.Size([14016, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3481])) that is different to the input size (torch.Size([3481, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/10], Train Loss: 0.1062, Val Loss: 0.1003
[0.00289799 0.00289799 0.00289799 ... 0.00289799 0.00289799 0.00289799]
